### 0. Расстояние Левенштейна

Моя реализация алгоритма коррекции на основе расстояния ливенштейна

Работает медленно, косо, криво и требуется более детального изучения вопроса, поэтому не будем изобретать велосипед,
продемонстрируем работу и пойдем к следующему пункту

In [16]:
#!pip install Levenshtein

In [7]:
import numpy as np
import requests
from Levenshtein import distance

In [8]:
response = requests.get('https://raw.githubusercontent.com/danakt/russian-words/master/russian.txt')

text = response.content.decode('cp1251')

with open('russian.txt', 'wb') as ru:
    ru.write(text.encode('utf-8'))

In [11]:
with open('russian.txt', encoding="utf-8") as file_in:
    dictionary = np.array([word.strip() for word in file_in.readlines()])

In [12]:
def get_candidates(word, dictionary):
    
    if len(word) == 1:
        print(f'Потеницальные кандидаты: {word} \nРасстояние Левенштейна = 0')
        return None
    
    distance_list = np.array([distance(user_word, word) for word in dictionary])
    lev_dist = distance_list.min()

    print(f'Потеницальные кандидаты: {dictionary[np.argwhere(distance_list == lev_dist)].T} \nРасстояние Левенштейна = {lev_dist}')

In [13]:
user_document = input()
for user_word in user_document.split():
    get_candidates(user_word, dictionary)

 сабака


Потеницальные кандидаты: [['абака' 'кабака' 'салака' 'слабака' 'собака' 'табака']] 
Расстояние Левенштейна = 1


### 1. Использование готового спеллчекера

Например это может быть pyspellchecker

из плюсов можно выделить поддержку русского языка из коробки

In [17]:
#!pip install pyspellchecker

In [18]:
from spellchecker import SpellChecker

spell = SpellChecker()
#document = ['Henlo', 'worldd', 'here', 'ss', 'farpost']
document = ['dogg']
misspelled = spell.unknown(document)

#answer = list()
for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

dog
{'dong', 'doggy', 'dog', 'dogs', 'doggo', 'doge'}


Получилось **что-то** и это **что-то** заработало

Сам алгоритм такого спеллчекинга был описан в этой [статье](https://norvig.com/spell-correct.html) Питером Норвигом, действующим
директором Гугл.

Этот алгоритм работает действительно хорошо, но попробуем сделать более интересную вещь и решить задау иначе...

### 2. Обучение машин

In [4]:
#!pip install transformers
#!pip install torch torchvision torchaudio
#!pip install transformers[sentencepiece]
#!pip install sentencepiece

In [4]:
#https://huggingface.co/ai-forever/RuM2M100-1.2B

In [6]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

path_to_model = "ai-forever/RuM2M100-1.2B"

model = M2M100ForConditionalGeneration.from_pretrained(path_to_model)
tokenizer = M2M100Tokenizer.from_pretrained(path_to_model, src_lang="ru", tgt_lang="ru")

sentence = "сабака"

encodings = tokenizer(sentence, return_tensors="pt")
generated_tokens = model.generate(
        **encodings, forced_bos_token_id=tokenizer.get_lang_id("ru"))
answer = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(answer)

#["прийдя в МГТУ я был удивлен никого не обнаружив там..."]

['собака']
